In [52]:
from app.rpd.api import RpdApi, Params
from app.rpd.RpdApp import CompetenceBoard
import json

In [53]:
api = RpdApi(data={
    Params.rup_row_id: 3019701,
    Params.rp_id: 741082,
})

In [54]:
comps_result = api.get_competencies_of_disciplines()
if comps_result.status_code == 200:
    comps = json.loads(comps_result.text)['data']['items']

GET https://rpd.donstu.ru/Rp3/Load {'rupRowId': 3019701, 'rpId': 741082}


In [55]:
cb = CompetenceBoard(None)
for c in comps:
    cb.add_item(c)

In [56]:
#cb.generate_table1('./table11.docx')

In [57]:
from docx import Document
from docx.enum.section import WD_ORIENT, WD_SECTION
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_CELL_VERTICAL_ALIGNMENT
from docx.enum.style import WD_STYLE_TYPE
from docx.shared import Cm
from docx.shared import Pt

from docx.table import _Cell
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

In [58]:
doc = Document()

In [59]:
def set_margins(section, top=None, bottom=None, left=None, right=None, all=None):
    if all is not None:
        if top is None: section.top_margin = Cm(all)
        if bottom is None: section.bottom_margin = Cm(all)
        if left is None: section.left_margin = Cm(all)
        if right is None: section.right_margin = Cm(all)
    if top is not None:
        section.top_margin = Cm(top)
    if bottom is not None:
        section.bottom_margin = Cm(bottom)
    if left is not None:
        section.left_margin = Cm(left)
    if right is not None:
        section.right_margin = Cm(right)


def rotate(section):
    h, w = section.page_height, section.page_width
    orient = section.orientation
    if orient == WD_ORIENT.PORTRAIT:
        section.orientation = WD_ORIENT.LANDSCAPE
    else:
        section.orientation = WD_ORIENT.PORTRAIT
    section.page_width = h
    section.page_height = w


def set_cell_border(cell: _Cell, **kwargs):
    """
    Set cell`s border
    Usage:
    set_cell_border(
        cell,
        top={"sz": 12, "val": "single", "color": "#FF0000", "space": "0"},
        bottom={"sz": 12, "color": "#00FF00", "val": "single"},
        start={"sz": 24, "val": "dashed", "shadow": "true"},
        end={"sz": 12, "val": "dashed"},
    )
    """
    tc = cell._tc
    tcPr = tc.get_or_add_tcPr()

    # check for tag existnace, if none found, then create one
    tcBorders = tcPr.first_child_found_in("w:tcBorders")
    if tcBorders is None:
        tcBorders = OxmlElement('w:tcBorders')
        tcPr.append(tcBorders)

    # list over all available tags
    for edge in ('start', 'top', 'end', 'bottom', 'insideH', 'insideV'):
        edge_data = kwargs.get(edge)
        if edge_data:
            tag = 'w:{}'.format(edge)

            # check for tag existnace, if none found, then create one
            element = tcBorders.find(qn(tag))
            if element is None:
                element = OxmlElement(tag)
                tcBorders.append(element)

            # looks like order of attributes is important
            for key in ["sz", "val", "color", "space", "shadow"]:
                if key in edge_data:
                    element.set(qn('w:{}'.format(key)), str(edge_data[key]))


def cell_center(cell):
    for p in cell.paragraphs:
        p.alignment = WD_TABLE_ALIGNMENT.CENTER
    cell.vertical_alignment = WD_CELL_VERTICAL_ALIGNMENT.CENTER


def cell_bold(cell):
    for p in cell.paragraphs:
        for r in p.runs:
            r.bold = True


def cell_border(cell):
    border = {"sz": 6, "color": "#000000", "val": "single"}
    set_cell_border(
        cell,
        top=border,
        bottom=border,
        start=border,
        end=border,
    )


def cell_body(cell):
    for p in cell.paragraphs:
        p.style = doc.styles['TableBodyStyle']
        p.paragraph_format.space_after = Pt(0)


def cell_header(cell):
    for p in cell.paragraphs:
        p.style = doc.styles['TableHeaderStyle']
        p.paragraph_format.space_after = Pt(0)


In [60]:
current_section = doc.sections[-1]
rotate(current_section)
set_margins(current_section, all=1, top=2)

In [61]:
headers = [
    {
        0: 'Компетенция',
        1: 'Запоминание',
        2: 'Оценочные средства',
        4: 'Понимание',
        5: 'Оценочные средства',
        7: 'Применение',
        8: 'Оценочные средства',
    },
    {
        2: 'текущий контроль',
        3: 'промежуточный контроль',
        5: 'текущий контроль',
        6: 'промежуточный контроль',
        8: 'текущий контроль',
        9: 'промежуточный контроль',
    }
]

cells_text = {
    2: '-',
    3: 'Ответы на вопросы №1-15',
    5: '-',
    6: 'Прохождение производственной практики. Оформление и защита отчета.',
    8: '-',
    9: 'Прохождение производственной практики. Оформление и защита отчета.',
}

styles = [

]
p_header_text = "Таблица 4 - Оценочные материалы (оценочные средства) по производственной практике «Ознакомительная практика»"

In [62]:
style = doc.styles.add_style('TableBodyStyle', WD_STYLE_TYPE.PARAGRAPH)
font = style.font
font.name = 'Times New Roman'
font.size = Pt(11)

style = doc.styles.add_style('TableHeaderStyle', WD_STYLE_TYPE.PARAGRAPH)
font = style.font
font.name = 'Times New Roman'
font.size = Pt(11)

In [63]:
table = doc.add_table(rows=0, cols=10)

for header in headers:
    row = table.add_row()
    for i in header:
        row.cells[i].text = header[i]

cols_count = len(table.rows[0].cells)
i = 0
while i < cols_count:
    ii = 1
    if i < cols_count - 1 and len(table.rows[0].cells[i + 1].text.strip()) == 0:
        table.rows[0].cells[i].merge(table.rows[0].cells[i + 1])
        ii += 1
    if len(table.rows[1].cells[i].text.strip()) == 0:
        table.rows[0].cells[i].merge(table.rows[1].cells[i])
    i += ii

for row in table.rows:
    for style in styles:
        if 'header' not in style['type']:
            continue
        for ind in style['indexes']:
            style['func'](row.cells[ind])

row = table.add_row()
row_index = len(table.rows) - 1

for competence in cb.competencies():
    for indicator in competence.indicators():
        
        last_index = row_index
        row = table.rows[row_index]
        
        row.cells[0].text = indicator.indi_code
        for level in indicator.levels():
            
# 
#         last_index = row_index
#         row = table.rows[row_index]
# 
#         row.cells[0].text = indicator.indi_code
#         row.cells[3].text = cells_text['type']
#         row.cells[4].text = cells_text['control']
#         row.cells[5].text = cells_text['OM']
#         row.cells[6].text = cells_text['crit']
# 
#         for level in indicator.levels():
#             if level.contents is None or len(level.contents) < 4:
#                 continue
#             row.cells[1].text = level_names[level.level_id % len(level_names)]
#             row.cells[2].text = level.contents
# 
#             row_index += 1
#             row = table.add_row()
# 
#         row1 = table.rows[last_index]
#         row2 = table.rows[row_index - 1]
# 
#         for row_id in range(last_index, row_index):
#             for style in styles:
#                 if 'body' not in style['type']:
#                     continue
#                 for ind in style['indexes']:
#                     style['func'](table.rows[row_id].cells[ind])
# 
#         for m_id in merge_cols:
#             row1.cells[m_id].merge(row2.cells[m_id])
# 
# row._element.getparent().remove(row._element)
# 
# for i in range(len(table.columns)):
#     for cell in table.columns[i].cells:
#         cell.width = Cm(cols_width[i])

Соединие (0,0) (1,0)
Соединие (0,1) (1,1)
Соединие (0,2) (0,3)
Соединие (0,4) (1,4)
Соединие (0,5) (0,6)
Соединие (0,7) (1,7)
Соединие (0,8) (0,9)


In [65]:
filepath = './table3.docx'
doc.save(filepath)